In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from dotenv import load_dotenv
import os

model_id = "meta-llama/Llama-3.2-1B-Instruct"
device = "cuda"  # Apple Silicon GPU

load_dotenv("keys.env")

# Get token from environment variables
token = os.getenv("token")

# Create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, token=token, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token
# Load model with float16 instead of bfloat16
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,  # Changed from bfloat16 to float16
    device_map="auto",  # Using "auto" instead of direct device mapping
    token=token
)


Some parameters are on the meta device because they were offloaded to the disk.


In [2]:
generation_pipeline = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
generation_pipeline("hello what are you?", max_new_tokens=25)

Device set to use mps


[{'generated_text': "hello what are you? \nI am a large language model, a computer program designed to understand and generate human-like text. I'm here to help"}]

In [9]:
input_prompt = [
  "Hello",
  "What is the capital of France?"
]

tokenised = tokenizer(input_prompt,padding=True,return_tensors="pt" ,truncation=True).to(device)
print(tokenised["input_ids"].shape)
tokenised["input_ids"]

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
tokenised["attention_mask"]

NameError: name 'tokenised' is not defined

In [ ]:
import torch

device = "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

mps


In [ ]:
prompt_template = [
    {
        "role": "system",
        "content": "You are a smart AI chatbot who speaks like a pirate."
    },
    {
        "role": "user",
        "content": "where does the sun rise?"
    },
    {
        "role": "assistant",
        "content": "Aye Aye"
    }
]

# Set pad token
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token_id = tokenizer.eos_token_id

tokenised = tokenizer.apply_chat_template(
    prompt_template,
    add_generation_prompt=False,
    continue_final_message=True,
    tokenize=True,
    padding=True,
    return_tensors="pt"
)
print(tokenised)
# Tokenize input
# tokenizer_output = tokenizer(
#     tokenizer.apply_chat_template(
#         prompt_template,
#         tokenize=False,
#         continue_final_message=True
#     ),
#     padding=True,
#     return_tensors="pt"
# )

# input_ids = tokenizer_output["input_ids"].to(device)
# attention_mask = tokenizer_output["attention_mask"].to(device)

# Generate text


tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1313,  13806,    220,   2366,     20,    271,   2675,    527,
            264,   7941,  15592,   6369,   6465,    889,  21881,   1093,    264,
          55066,     13, 128009, 128006,    882, 128007,    271,   2940,   1587,
            279,   7160,  10205,     30, 128009, 128006,  78191, 128007,    271,
             32,   9188,    362,   9188]])


In [60]:

out = model.generate(tokenised.to("mps"), max_new_tokens=25)
# outputs = model.generate(input_ids=input_ids,attention_mask=attention_mask, max_new_tokens=20)

# # Decode and print the response
# decoded = tokenizer.batch_decode(outputs)
# print(decoded[0])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [61]:
decoded = tokenizer.batch_decode(out)
print(decoded[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 22 Feb 2025

You are a smart AI chatbot who speaks like a pirate.<|eot_id|><|start_header_id|>user<|end_header_id|>

where does the sun rise?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Aye Aye Captain! Yer want ta know where the sun rises, eh? Well, matey, it's a mighty big question,


In [7]:
import json
import pandas as pd

df = pd.read_csv("data_output_files/summary_data.csv")
df.head()

,title,url,category,final_score,impact,scale,potential,legacy,novelty,credibility,positivity
0,Biden Says U.S. Forces Would Defend Taiwan If ...,https://www.huffpost.com/entry/biden-us-forces...,POLITICS,6.7,7,8,8,6,5,7,3
1,‘Beautiful And Sad At The Same Time’: Ukrainia...,https://www.huffpost.com/entry/ukraine-festiva...,POLITICS,4.2,4,4,5,3,4,8,5
2,Biden Says Queen's Death Left 'Giant Hole' For...,https://www.huffpost.com/entry/europe-britain-...,POLITICS,4.0,3,4,3,6,3,7,5
3,Bill To Help Afghans Who Escaped Taliban Faces...,https://www.huffpost.com/entry/afghan-adjustme...,POLITICS,3.8,3,4,4,3,4,7,5
4,Mark Meadows Complies With Justice Dept. Subpo...,https://www.huffpost.com/entry/capitol-riot-in...,POLITICS,4.5,4,4,5,5,5,7,2


In [8]:
# from .openailabeling import NewsSignificanceCalculator
# NSC = NewsSignificanceCalculator()
# NSC.extract_article_content(df.iloc[0]["url"])
import os
print(os.getcwd())
print(os.listdir())
import openailabeling
from openailabeling import NewsSignificanceCalculator
import importlib
importlib.reload(openailabeling)


/Users/naeemujeeb/PycharmProjects/GlobalHQ/GlobalLEARN
['fetch_article.py', 'prompt_files', 'requirements.txt', 'environment.yml', 'Dockerfile', 'dataset', 'sample_articles', '__pycache__', 'openailabeling.py', 'model1.ipynb', 'data_output_files', 'keys.env', 'main.py']


<module 'openailabeling' from '/Users/naeemujeeb/PycharmProjects/GlobalHQ/GlobalLEARN/openailabeling.py'>

In [9]:
from openailabeling import NewsSignificanceCalculator
NSC = NewsSignificanceCalculator()

SYSTEM_PROMPT = {
    "role": "system",
    "content": """Return a score of each factor for the news article I have attached at the end (0-10 for each factor):

Factors and weights:
- scale : Global population impact (10=affects all humanity, 5=regional, 2=local)
- impact : Immediate effect strength on humanity's development
- potential : Future influence on human civilization
- legacy : Historical milestone for human progress
- novelty : Uniqueness in human history
- credibility : Source reliability
- positivity : Positive development for humanity

Scale scoring guideline:
10: Affects all of humanity directly
8: Major global impact
6: Multi-region impact
4: Regional impact
2: Local impact

Impact scoring guideline:
10: Immediate global emergency
8: Major global change
6: Significant regional change
4: Moderate regional effect
2: Local effect

Return ONLY a JSON object with scores for the following news article. Use this format:
{
    "scale": N,
    "impact": N,
    "potential": N,
    "legacy": N,
    "novelty": N,
    "credibility": N,
    "positivity": N
}


Calibration Examples:
Global Impact (6.3/10): "Earth surpasses 1.5°C warming limit"
{
    "scale": 9,
    "impact": 8,
    "potential": 9,
    "legacy": 8,
    "novelty": 7,
    "credibility": 9,
    "positivity": 2
}

Regional Conflict (5.0/10): "Military escalation in ongoing war"
{
    "scale": 5,
    "impact": 6,
    "potential": 5,
    "legacy": 5,
    "novelty": 5,
    "credibility": 9,
    "positivity": 2
}

Local Event (2.2/10): "Regional sports championship"
{
    "scale": 2,
    "impact": 2,
    "potential": 1,
    "legacy": 1,
    "novelty": 2,
    "credibility": 8,
    "positivity": 8
}

Significance ranges:
High (6+): Major impact on human civilization
Medium (3-5): Regional/industry significance
Low (1-2): Local/minor impact
Score based on the following news article
"""
}

# Create list of user messages for each article
user_messages = [
    {
        "role": "user",
        "content": f"URL: {row['url']}\nHeadline: {row['title']}\nContent: {NSC.extract_article_content(row['url']).get('text', '')}"
    }
    for _, row in df.iloc[:5].iterrows()
]

# Create complete prompt for each article
PROMPTS = [
    [SYSTEM_PROMPT, user_message]
    for user_message in user_messages
]


removing ads and new lines
removing ads and new lines
removing ads and new lines
removing ads and new lines
removing ads and new lines


In [10]:
print(SYSTEM_PROMPT["content"])

Return a score of each factor for the news article I have attached at the end (0-10 for each factor):

Factors and weights:
- scale : Global population impact (10=affects all humanity, 5=regional, 2=local)
- impact : Immediate effect strength on humanity's development
- potential : Future influence on human civilization
- legacy : Historical milestone for human progress
- novelty : Uniqueness in human history
- credibility : Source reliability
- positivity : Positive development for humanity

Scale scoring guideline:
10: Affects all of humanity directly
8: Major global impact
6: Multi-region impact
4: Regional impact
2: Local impact

Impact scoring guideline:
10: Immediate global emergency
8: Major global change
6: Significant regional change
4: Moderate regional effect
2: Local effect

Return ONLY a JSON object with scores for the following news article. Use this format:
{
    "scale": N,
    "impact": N,
    "potential": N,
    "legacy": N,
    "novelty": N,
    "credibility": N,
   

In [15]:
print(user_messages[0]["content"])

URL: https://www.huffpost.com/entry/biden-us-forces-defend-taiwan-against-china_n_6327ec68e4b0ed021dfe3695
Headline: Biden Says U.S. Forces Would Defend Taiwan If China Invaded
Content:  BEIJING (AP) — President Joe Biden says U.S. forces would defend Taiwan if China tries to invade the self-ruled island claimed by Beijing as part of its territory, adding to displays of official American support for the island democracy. Biden said “yes” when asked during an interview broadcast Sunday on CBS News’s “60 Minutes” program whether “U.S. forces, U.S. men and women, would defend Taiwan in the event of a Chinese invasion.”  CBS News reported the White House said after the interview U.S. policy hasn’t changed. That policy says Washington wants to see Taiwan’s status resolved peacefully but doesn’t say whether U.S. forces might be sent in response to a Chinese attack. Tension is rising following efforts by Chinese President Xi Jinping’s government to intimidate Taiwan by firing missiles into th

In [47]:
print(PROMPTS)

[[{'role': 'system', 'content': 'Return a score of each factor for the news article I have attached at the end (0-10 for each factor):\n\nFactors and weights:\n- scale : Global population impact (10=affects all humanity, 5=regional, 2=local)\n- impact : Immediate effect strength on humanity\'s development\n- potential : Future influence on human civilization\n- legacy : Historical milestone for human progress\n- novelty : Uniqueness in human history\n- credibility : Source reliability\n- positivity : Positive development for humanity\n\nScale scoring guideline:\n10: Affects all of humanity directly\n8: Major global impact\n6: Multi-region impact\n4: Regional impact\n2: Local impact\n\nImpact scoring guideline:\n10: Immediate global emergency\n8: Major global change\n6: Significant regional change\n4: Moderate regional effect\n2: Local effect\n\nReturn ONLY a JSON object with scores for the following news article. Use this format:\n{\n    "scale": N,\n    "impact": N,\n    "potential": 

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenised_output = tokenizer(
  tokenizer.apply_chat_template(
    PROMPTS,
    tokenize=False,
    continue_final_message=True
  ),
  padding=True,
  return_tensors="pt"
)

input_ids = tokenised_output["input_ids"].to(device)
attention_mask = tokenised_output["attention_mask"].to(device)

# tokenised = tokenizer.apply_chat_template(
#   PROMPTS,
#   continue_final_message=True,
#   padding=True,
#   return_tensors="pt"
# ).to(device)
out = model.generate(input_ids=input_ids, attention_mask=attention_mask,  max_new_tokens=50)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [49]:
decoded = tokenizer.batch_decode(out, skip_special_tokens=True)
print(decoded)

['system\n\nCutting Knowledge Date: December 2023\nToday Date: 24 Feb 2025\n\nReturn a score of each factor for the news article I have attached at the end (0-10 for each factor):\n\nFactors and weights:\n- scale : Global population impact (10=affects all humanity, 5=regional, 2=local)\n- impact : Immediate effect strength on humanity\'s development\n- potential : Future influence on human civilization\n- legacy : Historical milestone for human progress\n- novelty : Uniqueness in human history\n- credibility : Source reliability\n- positivity : Positive development for humanity\n\nScale scoring guideline:\n10: Affects all of humanity directly\n8: Major global impact\n6: Multi-region impact\n4: Regional impact\n2: Local impact\n\nImpact scoring guideline:\n10: Immediate global emergency\n8: Major global change\n6: Significant regional change\n4: Moderate regional effect\n2: Local effect\n\nReturn ONLY a JSON object with scores for the following news article. Use this format:\n{\n    "sc

In [ ]:
def extract_label(decoded_text: str) -> str:
    """Extract the label from model output text"""
    try:
        # Split at the start token
        parts = decoded_text.split("<|start_header_id|>assistant<|end_header_id|>")
        if len(parts) < 2:
            return None

        # Get the part after the header
        content = parts[1]

        # Split at the end token and get the first part
        label = content.split("<|eot_id|>")[0].strip()

        return label

    except Exception as e:
        print(f"Error extracting label: {e}")
        return None

# Use it on your list
labels = []
for text in decoded:
    label = extract_label(text)
    if label:
        labels.append(label)

print(labels)

[]


In [35]:
first_row = df.iloc[0]
first_row
PROMPT = [
  {
    "role" : "system",
    "content" : SYSTEM_PROMPT["content"]
  },
  {
    "role" : "user",
    "content": f"URL: {first_row['url']}\nHeadline: {first_row['title']}\nContent: {NSC.extract_article_content(first_row['url']).get('text', '')}"
  }
]



removing ads and new lines


In [10]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id


tokenised = tokenizer.apply_chat_template(
  PROMPT,
  continue_final_message=True,
  padding=True,
  return_tensors="pt"
).to(device)
input_ids = tokenised_output["input_ids"].to(device)
attention_mask = tokenised_output["attention_mask"].to(device)

out = model.generate(input_ids=input_ids, attention_mask=attention_mask,  max_new_tokens=50)

NameError: name 'PROMPT' is not defined

In [2]:
decoded = tokenizer.batch_decode(out, skip_special_tokens=True)
decoded

NameError: name 'tokenizer' is not defined

In [11]:
import torch as nn
# the text
print(torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"

text = "Hello how are"
# tokenising the text
input_ids = tokenizer([text], return_tensors="pt")["input_ids"].to(device)
# pass into model
out = model(input_ids=input_ids, max_new_tokens=50)


False


In [ ]:
#128000, 9906, 1268, 527 these are the tokens, 128000 is the beginning of a sentence, 9906 is the Hello etc
input_ids

tensor([[128000,   9906,   1268,    527]])

In [12]:
import torch.nn as nn
probabilitydist = nn.Softmax()(out.logits[0, -1])


/opt/anaconda3/envs/llmspace/lib/python3.9/site-packages/torch/nn/modules/module.py:1739: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [13]:
# probability of the next token being 450 from the model
probabilitydist[499]

tensor(0.9883, dtype=torch.bfloat16, grad_fn=<SelectBackward0>)

In [27]:
# unnormalised scores that the model produces as its confidence in what it thinks the next word will be
out.logits.shape

torch.Size([1, 4, 128256])

In [29]:
out.logits[0, -1]

tensor([9.8750, 6.3125, 1.7266,  ..., 0.4023, 0.4023, 0.4004],
       dtype=torch.bfloat16, grad_fn=<SelectBackward0>)

In [15]:
tokenizer.convert_ids_to_tokens(9514)
# there is a space before you and space after you
# this calculates the token value of the word "you"
tokenizer.vocab["you"]
tokenizer.vocab["you"]

9514

In [54]:
out.logits.argmax(axis=-1)[0, -1]

tensor(499)

### Training on sequence

tensor([[128000,  29673,    311,  61577,   8304,    449,  12431,     33]])
['<|begin_of_text|>Subscribe to Neural Network with AVB']


In [ ]:
# We input this into the transformer
# Then it will apply its causual mask attention
# And then the output token will be what it predicts as the next token
# extract the input sequence and target sequence from the tokenised sequence
input_ids = tokenised[:, :-1] # start to end - 1
target_ids = tokenised[:, 1:] # start + 1 to end

print(input_ids)
print(target_ids)


tensor([[128000,  29673,    311,  61577,   8304,    449,  12431]])
tensor([[29673,   311, 61577,  8304,   449, 12431,    33]])


In [ ]:
# Loss functions
sentence

In [1]:
sentence = ["Subscribe to neural network break with AVB"]
tokenized = tokenizer(sentence, return_tensors="pt")["input_ids"]
print(tokenized)

NameError: name 'tokenizer' is not defined

In [17]:
#
sentence = ["Subscribe to Neural Network with AVB"]
tokenised = tokenizer(sentence, return_tensors="pt")["input_ids"]
print(tokenised)
print(tokenizer.batch_decode(tokenised))


tensor([[128000,  29673,    311,  61577,   8304,    449,  12431,     33]])
['<|begin_of_text|>Subscribe to Neural Network with AVB']


### Finetuning sentences

In [ ]:
#tokenising the sentence
sentence = ["Global warming temperature increase by 2 degrees"]
tokenised = tokenizer(sentence, return_tensors="pt")["input_ids"]
print(tokenised)
print(tokenizer.batch_decode(tokenised))

tensor([[128000,  11907,  24808,   9499,   5376,    555,    220,     17,  12628]])
['<|begin_of_text|>Global warming temperature increase by 2 degrees']


In [ ]:
# extracting input and target from the sequence
#input into transformer
input_ids = tokenised[:, :-1]
# apply causal mask attention
# output token will be what it predicts as the next token
target_ids = tokenised[:, 1:]

print(input_ids)
print(target_ids)

tensor([[128000,  11907,  24808,   9499,   5376,    555,    220,     17]])
tensor([[11907, 24808,  9499,  5376,   555,   220,    17, 12628]])


In [24]:
question = "Capital of Delhi?"
answer = "New Delhi"

prompt = [
  {"role" : "user", "content": "Capital of India?"},
  {"role" : "assistant", "content": "Capital:"}
]

chat_template = tokenizer.apply_chat_template(prompt, continue_final_message=True, tokenize=False)
print(chat_template)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 25 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Capital of India?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Capital:


In [ ]:
#the entire sequence that we use so the model can learn on its won
full_response = chat_template + " " + answer + tokenizer.eos_token
print(full_response)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 25 Feb 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Capital of India?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Capital: New Delhi<|eot_id|>


In [27]:
tokenised = tokenizer(full_response, return_tensors="pt", add_special_tokens=False)["input_ids"]
print(tokenised)

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,    914,  13806,    220,   2366,     20,    271, 128009, 128006,
            882, 128007,    271,  64693,    315,   6890,     30, 128009, 128006,
          78191, 128007,    271,  64693,     25,   1561,  22767, 128009]])


In [ ]:
#input and target shifted by one to calculate the loss
input_ids = tokenised[:, :-1]
target_ids = tokenised[:, 1:]

print(input_ids)
print(target_ids)

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,    914,  13806,    220,   2366,     20,    271, 128009, 128006,
            882, 128007,    271,  64693,    315,   6890,     30, 128009, 128006,
          78191, 128007,    271,  64693,     25,   1561,  22767]])
tensor([[128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,     25,
           6790,    220,   2366,     18,    198,  15724,   2696,     25,    220,
            914,  13806,    220,   2366,     20,    271, 128009, 128006,    882,
         128007,    271,  64693,    315,   6890,     30, 128009, 128006,  78191,
         128007,    271,  64693,     25,   1561,  22767, 128009]])


In [ ]:
# So target is the full sequence with New Delhi, and we calcualte the loss between input
# and the target, so the model can learn to predict the next token which is New Delhi
# It calculates the loss which is the difference between the predicted token and the actual token
# It then backpropagates the loss to the model
# The model then updates its weights to reduce the loss
# The model then predicts the next token
# The model then calculates the loss
# The model then backpropagates the loss
# The model then updates its weights

labels_tokenised = tokenizer([" " + answer], add_special_tokens=False, return_tensors="pt")["input_ids"]
print(labels_tokenised)


tensor([[ 1561, 22767]])


In [32]:
labels_tokenised = tokenizer([" " + answer + tokenizer.eos_token],
           add_special_tokens=False, return_tensors="pt", padding="max_length",
           max_length=target_ids.shape[1])["input_ids"]

In [ ]:
#We want padding, padding which will be replaced by the actual answer
#So the answer is 1561, 22767 and 128009 and the rest is padding
labels_tokenised

tensor([[128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009,   1561,  22767, 128009]])

In [ ]:
# We want to replace the padding with -100
# this is recommended by PyTorch for the loss function
# -100 is used to mask the padding
# so the model does not learn from the padding
# the model only learns from the actual tokens
labels_tokenised_fixed = torch.where(labels_tokenised != tokenizer.pad_token_id, labels_tokenised, -100)
print(labels_tokenised_fixed)

tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          1561, 22767,  -100]])


In [2]:


def generate_input_output_pair(prompt, target_response):
    # Apply chat template
    chat_template = tokenizer.apply_chat_template(prompt, continue_final_message=True, tokenize=False)
    # Tokenize input sequence
    input_sequence = tokenizer(chat_template, return_tensors="pt", add_special_tokens=False)["input_ids"]

    # Tokenize target sequence
    target_sequence = tokenizer(" " + target_response + tokenizer.eos_token,
                                return_tensors="pt", add_special_tokens=False)["input_ids"]

    # Get full sequence
    full_sequence = torch.cat([input_sequence, target_sequence], dim=1)

    # Create inputs (without the last token)
    inputs = full_sequence[:, :-1]
    attention_mask = inputs != tokenizer.pad_token_id

    # Create labels: -100 for input tokens, actual ids for target tokens
    labels = torch.full_like(inputs, -100)
    target_len = target_sequence.shape[1]
    labels[:, -target_len+1:] = full_sequence[:, -target_len:][:, 1:]  # Shift target right by 1

    return {
        "input_ids": inputs,
        "attention_mask": attention_mask,
        "labels": labels
    }

prompt = [
  {"role" : "user", "content": "Capital of India?"},
  {"role" : "assistant", "content": "Capital:"}
]
answer = "Mumbai"
data = generate_input_output_pair(prompt, answer)
print(data)


{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  13806,    220,   2366,     20,    271, 128009, 128006,
            882, 128007,    271,  64693,    315,   6890,     30, 128009, 128006,
          78191, 128007,    271,  64693,     25,  35812]]), 'attention_mask': tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True, False,  True,  True,  True,  True,
          True,  True,  True,  True, False,  True,  True,  True,  True,  True,
          True,  True]]), 'labels': tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -10

In [3]:
data["input_ids"]

tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  13806,    220,   2366,     20,    271, 128009, 128006,
            882, 128007,    271,  64693,    315,   6890,     30, 128009, 128006,
          78191, 128007,    271,  64693,     25,  35812]])

In [4]:
# The labels are the actual tokens that the model should predict
data["labels"]

tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100, 128009]])

In [55]:
out = model(input_ids=data["input_ids"])

In [56]:
out.logits.shape

torch.Size([1, 42, 128256])

In [57]:
data["labels"]

tensor([[  -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100, 128009]])

In [17]:
import torch.nn as nn
def calculate_loss(logits, labels):
  loss_fn = nn.CrossEntropyLoss(reduction='none')
  cross_entropy_loss = loss_fn(logits.view(-1, logits.shape[-1]), labels.view(-1))
  return cross_entropy_loss

# loss = calculate_loss(out.logits, data["labels"].to(device))
# loss

In [8]:
training_prompt = [
  {
    "role" : "user",
    "content": "Score a impact of the news article out of 10 : Global Tmeperature Increase by 2 degrees"
  },
  {
    "role" : "assistant",
    "content": "Significance Score :"
  }

]

target_response = "6.3/10"

In [12]:
device = "cuda"
test_tokenized = tokenizer.apply_chat_template(training_prompt, continue_final_message=True, return_tensors="pt")
test_out = model.generate(test_tokenized, max_new_tokens=10)
print(tokenizer.batch_decode(test_out, skip_special_tokens=True)[0])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/opt/anaconda3/envs/llmspace/lib/python3.9/site-packages/transformers/generation/utils.py:2105: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('mps') before running `.generate()`.
  warnings.warn(


system

Cutting Knowledge Date: December 2023
Today Date: 26 Feb 2025

user

Score a impact of the news article out of 10 : Global Tmeperature Increase by 2 degreesassistant

Significance Score : 7

The news article reports on the global


In [20]:
from transformers import AdamW
#
data = generate_input_output_pair(training_prompt, target_response)
data["input_ids"] = data["input_ids"]
data["labels"]

optimiser = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

for _ in range(10):
    out = model(input_ids=data["input_ids"])
    loss = calculate_loss(out.logits, data["labels"]).mean()

    loss.backward()
    optimiser.step()
    optimiser.zero_grad()

RuntimeError: MPS backend out of memory (MPS allocated: 7.17 GB, other allocations: 1.54 GB, max allowed: 9.07 GB). Tried to allocate 501.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).